# Model Optimization

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn import cluster
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import tree

In [2]:
iris_df = sns.load_dataset('iris')


iris_X = iris_df.drop(columns=['species'])

iris_y = iris_df['species']

In [3]:
iris_X_train, iris_X_test, iris_y_train, iris_y_test = train_test_split(iris_X,iris_y)

In [4]:
dt = tree.DecisionTreeClassifier()
params_dt = {'criterion':['gini','entropy'],'max_depth':[2,3,4],
       'min_samples_leaf':list(range(2,20,2))}

In [5]:
dt_opt = GridSearchCV(dt,params_dt,cv=10)

In [6]:
dt_opt.fit(iris_X_train,iris_y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4],
                         'min_samples_leaf': [2, 4, 6, 8, 10, 12, 14, 16, 18]})

In [7]:
y_pred = dt_opt.predict(iris_X_test)

In [8]:
dt_opt.score(iris_X_test,iris_y_test)

0.9736842105263158

In [9]:
dt_opt.best_params_

{'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 4}

In [10]:
pd.DataFrame(dt_opt.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002921,0.000839,0.002038,0.000460,gini,2,2,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
1,0.002600,0.000482,0.001795,0.000244,gini,2,4,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
2,0.002409,0.000310,0.001776,0.000267,gini,2,6,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
3,0.002556,0.000303,0.001787,0.000150,gini,2,8,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
4,0.002290,0.000128,0.001625,0.000112,gini,2,10,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
5,0.002603,0.000561,0.001753,0.000239,gini,2,12,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
6,0.002493,0.000264,0.001830,0.000584,gini,2,14,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
7,0.002248,0.000461,0.001777,0.000355,gini,2,16,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
8,0.002164,0.000117,0.001511,0.000110,gini,2,18,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,...,0.909091,0.818182,1.0,0.909091,1.0,1.0,0.909091,0.919697,0.073621,11
9,0.002529,0.000203,0.001731,0.000168,gini,3,2,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.750000,1.0,...,0.909091,0.818182,1.0,1.000000,1.0,1.0,1.000000,0.929545,0.093403,5


In [11]:
from sklearn import model_selection

In [12]:
model_selection.RandomizedSearchCV()

TypeError: __init__() missing 2 required positional arguments: 'estimator' and 'param_distributions'